<h1>Project Data Mining</h1>
<br><h2>Sentiment Analysis on tweets</h2>
<br><b>Editors: </b>
<br>Aslanidis Theofanis - Athinaios Konstantinos

<b>1)</b> Importing all needed packages: <br>
<ul>
    <li><b>nltk:</b> used for tokenization
    <li><b>pandas:</b> data representation
    <li><b>WordCloud:</b> data representation
<ul>


In [92]:
import numpy as np
import pandas as pd
import matplotlib as plt
from nltk.tokenize import RegexpTokenizer, word_tokenize, TweetTokenizer
from wordcloud import WordCloud

#nltk.download('punkt')
#nltk.download('stopwords')

<b>2)</b> Loading tweeter datasets for training and testing:
<ul>
    <li><u>Value</u> for negative-positive-neutral
    <li><u>Tweet</u> for the text
</ul>

In [93]:
train_data = pd.read_csv("../twitter_data/train2017.tsv", names=['Value', 'Tweet'], sep='\t', usecols = [2, 3])
test_data = pd.read_csv("../twitter_data/test2017.tsv", names=['Value', 'Tweet'], sep='\t', usecols = [2, 3])
train_data.head()

,Value,Tweet
0,positive,Gas by my house hit $3.39!!!! I'm going to Cha...
1,negative,"Theo Walcott is still shit, watch Rafa and Joh..."
2,negative,"its not that I'm a GSP fan, i just hate Nick D..."
3,negative,Iranian general says Israel's Iron Dome can't ...
4,neutral,"Tehran, Mon Amour: Obama Tried to Establish Ti..."


<h4>Our pipeline:</h4><br>
<ol>
    <li>Load the raw text.
    <li>Split into tokens.
    <li>Convert to lowercase.
    <li>Filter out tokens that are stop words.
    <li>Stemming
</ol>
<br>    
<b>Tokenize tweets and convert all to lowercase</b>


In [124]:
tweets = []
tokenizer = TweetTokenizer()
for tweet in train_data.T:
    tweets.append(tokenizer.tokenize(train_data.T[tweet][1].lower()))

<b>Remove Stop Words</b>

In [125]:
# filter out stop words
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
for x in range(0,len(tweets)):
    tweets[x] = [token for token in tweets[x] if not token in stop_words]

<b>Stemming (Reducing each word to its root).</b>

In [126]:
# stemming of words
from nltk.stem.porter import PorterStemmer

stemmed = []
porter = PorterStemmer()
for x in range(0,len(tweets)):
    stemmed.append([porter.stem(token) for token in tweets[x]])

In [129]:
print(tweets[0], "\n", stemmed[0])

['gas', 'house', 'hit', '$', '3.39', '!', '!', '!', "i'm", 'going', 'chapel', 'hill', 'sat', '.', ':)'] 
 ['ga', 'hous', 'hit', '$', '3.39', '!', '!', '!', "i'm", 'go', 'chapel', 'hill', 'sat', '.', ':)']


<h2>Data Analysis</h2>
<br><h4>Characteristics:</h4>
<ul>
    <li><b>Most Frequent</b> words in whole training dataset.
    <li><b>Most Frequent Positive, Negative and Neutral </b> words.
<ul>